In [26]:
# Merge nas 5 tabelas distintas para trabalharmos com uma Dataframe única

import pandas as pd
from pyxlsb import open_workbook as open_xlsb

MD1 = pd.read_excel('DadosOTs_Transformados - Copia PoliJr.xlsb', sheet_name='MasterData1')

MD2 = pd.read_excel('DadosOTs_Transformados - Copia PoliJr.xlsb', sheet_name='MasterData2')
MD2.rename(columns={'TurbineFunctionalLocation': 'Aerogenerador'}, inplace=True)

OTs = pd.read_excel('DadosOTs_Transformados - Copia PoliJr.xlsb', sheet_name='Consumo_OTs')
OTs.rename(columns={'Aerogenerador (Ubicación Técnica)': 'Aerogenerador'}, inplace=True)

MDT1 = pd.read_excel('DadosOTs_Transformados - Copia PoliJr.xlsb', sheet_name='MasterData_Materiais1')
MDT1.rename(columns={'Number': 'Material'}, inplace=True)

MDT2 = pd.read_excel('DadosOTs_Transformados - Copia PoliJr.xlsb', sheet_name='MasterData_Materiais2')

df = MD1.merge(MD2, on='Aerogenerador', how='inner')
df = df.merge(OTs, on='Aerogenerador', how='inner')
df = df.merge(MDT1, on='Material', how='inner')
df = df.merge(MDT2, on='Material', how='inner')
df

df.to_excel('Merge_Dados.xlsx')

In [45]:
import pandas as pd

def process_table(df):
    result_rows = []
    zero_965_nonzero_966 = []
    
    # Itera por cada ORDEN única
    for order in df['Orden'].unique():
        df_order = df[df['Orden'] == order]
        
        # Itera por cada MATERIAL único dentro da ORDEN
        for material in df_order['Material'].unique():
            df_material = df_order[df_order['Material'] == material]
            
            # Soma as quantidades para as classes de movimento 965 e 966
            qty_965 = df_material[df_material['Clase de movimiento'] == 965]['Cantidad'].sum()
            qty_966 = df_material[df_material['Clase de movimiento'] == 966]['Cantidad'].sum()
            
            # Calcula o saldo
            saldo = qty_965 - qty_966
            
            # Cria uma nova linha com os dados agregados
            new_row = {
                'Orden': order,
                'Material': material,
                'Clase de movimiento': 965,
                'Saldo': saldo
            }
            
            result_rows.append(new_row)
            
            # Verifica se qty_965 é zero e qty_966 é diferente de zero
            if qty_965 == 0 and qty_966 != 0:
                zero_965_nonzero_966.append({
                    'Orden': order,
                    'Material': material,
                    'Clase de movimiento': 966,
                    'Cantidad 966': qty_966
                })
    
    # Cria um novo DataFrame com as linhas resultantes
    df_result = pd.DataFrame(result_rows)
    df_zero_965_nonzero_966 = pd.DataFrame(zero_965_nonzero_966)
    
    return df_result, df_zero_965_nonzero_966

# Carregar o dataframe original
df = pd.read_excel('DadosOTs_Transformados - Copia PoliJr.xlsb', sheet_name='Consumo_OTs')

# Processar o dataframe
df_processed, df_zero_965_nonzero_966 = process_table(df)

# Salvar o dataframe processado em um novo arquivo
df_processed.to_excel('Balanco_Compras.xlsx', index=False)

# Salvar o dataframe com qty_965 zero e qty_966 diferente de zero em um novo arquivo
df_zero_965_nonzero_966.to_excel('Linhas_Zero_965_Nonzero_966.xlsx', index=False)

# Imprimir o dataframe e o número de linhas
print(df_zero_965_nonzero_966)
print(f"Number of rows: {df_zero_965_nonzero_966.shape[0]}")


          Orden  Material  Clase de movimiento  Cantidad 966
0     402606584  GP082781                  966           1.0
1     402612935  GP011546                  966           1.0
2     402618103     50846                  966           1.0
3     402624587  GP266727                  966           1.0
4     402624587  GP105540                  966           1.0
...         ...       ...                  ...           ...
4960  405156145   AMB0155                  966           1.0
4961  405161328  GP010564                  966           1.0
4962  405161481  GP004265                  966           1.0
4963  405163832  GP004891                  966           1.0
4964  405167591  GP234915                  966           1.0

[4965 rows x 4 columns]
Number of rows: 4965


In [39]:
print(df_processed[df_processed['Cantidad'] < 0])

            Orden  Material  Clase de movimiento  Cantidad
44      402606584  GP082781                  965      -1.0
121     402612935  GP011546                  965      -1.0
173     402618103     50846                  965      -1.0
236     402624050  GP011692                  965      -1.0
246     402624587  GP266727                  965      -1.0
...           ...       ...                  ...       ...
155802  405156145   AMB0155                  965      -1.0
155934  405161328  GP010564                  965      -1.0
155958  405161481  GP004265                  965      -1.0
156074  405163832  GP004891                  965      -1.0
156114  405167591  GP234915                  965      -1.0

[6106 rows x 4 columns]
